Importa biblioteca Pandas, chama de 'pd' e libera o acesso ao Google Drive

In [2]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import math

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Configura o pandas para mostrar todas as colunas do dataframe
pd.options.display.max_columns = None


Mounted at /content/drive


Função que trata um arquivo CSV e passa para dataframe

In [3]:
def preparaDF(caminho_para_pasta,caminho_para_arquivo):
  # O arquivo apresentou erros de separador e codificação.
  # Os dados começam na linha 12
  # Não usa coluna como índice
  df = pd.read_csv(f'{caminho_para_pasta}{caminho_para_arquivo}', sep=';', encoding="ISO-8859-1", skiprows = 12, index_col=False, decimal=',')
  #print(df.columns)
  # Muda o tipo da coluna 'Data' para datetime (string --> datetime)
  df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')

  # A coluna 'Data' e 'NivelConsistencia' são usadas como índice
  df.set_index(['Data','NivelConsistencia'], inplace=True)

  # Exclui colunas desnecessárias do dataframe
  listColumnsDrop = []
  for i in range(73):
    if i > 41:
      listColumnsDrop += [i]
  listColumnsDrop += [0,1,2,3,4,5,6,7,8,9,10]
  df = df.drop(df.columns[listColumnsDrop], axis=1)

  # Avalia se existe uma mesma data com diferentes níveis de consistência
  # Classificar o dataframe por data
  df.sort_values("Data", inplace=True)

  # Reseta os índices para passar "Data" como coluna e utilizar a mesma para apagar as linhas duplicadas
  df = df.reset_index()

  # apagas as linhas duplicadas, dando preferência ao dado consistido ('NivelConsistencia' = 2)
  df = df.drop_duplicates(subset='Data', keep='last', inplace=False)

  # A coluna 'Data' e 'NivelConsistencia' são usadas como índice
  df.set_index(['Data','NivelConsistencia'], inplace=True)

  # transforma as colunas de chuva em linhas com as respectivas datas
  # Este processo transforma todas as colunas em linhas, gerando um problema para os meses que possuem menos de 31 dias
  # Dessa forma, o dia 1 de um mês armazenava a chuva do dia 1 e a chuva do dia 31 (linha duplicada com valor NaN)
  # Correção explicada durante o processo:

  # usa a função .melt() para transformar todas as colunas em linhas. São mantidas as colunas ['Data', 'NivelConsistencia']
  # São criadas duas novas colunas: 'Dia' que recebe o nome da coluna que virou linha e 'Chuva' que recebe o valor que era armazenado na coluna
  df = df.reset_index().melt(id_vars=['Data', 'NivelConsistencia'], var_name='Dia', value_name='Chuva')

  # Fiz uma cópia da minha data em formato de texto
  df['Data1']=df['Data'].astype(str)

  # Apaguei os últimos 2 caracteres, assim ficou somente o texto com %Y-%m- ('1989-11-')
  df['Data1'] = df['Data1'].str[:-2]

  # Fiz uma cópia da coluna 'Dia' (exe: 'Chuva03'), a 'função .str.extract('(\d+)', expand=False)' deixa apenas os números
  # (então a coluna 'Dia1' armazena a string '03') ('Chuva03' --> '03')  
  df['Dia1'] = df['Dia'].str.extract('(\d+)', expand=False)

  # praticamente a mesma coisa da linha anterior, mudando apenas que substitui a própria coluna e o formato é int ('Chuva03' --> 3)
  df['Dia'] = df['Dia'].str.extract('(\d+)', expand=False).astype(int)

  # Como foram criadas várias linhas, tem vários índices repetidos da data, esta função soma os dias correspondente
  # aos da chuva guardado na coluna 'Dia' na data do índice (aqui é gerado o erro dos 31 dias)
  df['Data'] = df.apply(lambda x: x['Data'] + pd.DateOffset(days=x['Dia']-1), axis=1)

  # Aqui é feita uma concatenação do tipo '1989-11-'+'03' formando uma data.
  # Mas aqui também forma datas como '1989-02-31'
  df['Data1'] = df['Data1'] + df['Dia1']

  # A maioria das datas das colunas 'Data' e 'Data1' serão iguais, mas datas como '1989-02-31'
  # vão gerar valor NaT pois é um erro (exatamente nas linhas duplicadas)
  df['Data1'] = pd.to_datetime(df['Data1'], format='%Y-%m-%d', errors='coerce')

  # Aqui é apagada todas as linhas em que o erro NaT aparece na coluna 'Data1' (removendo os valores duplicados)
  df.dropna(inplace=True, subset=['Data1'])

  # reorganiza as colunas
  df = df[['Data', 'NivelConsistencia', 'Chuva']]

  # Classificar o dataframe por data
  df.sort_values("Data", inplace=True)

  # A coluna 'Data' e 'NivelConsistencia' são usadas como índice novamente
  df.set_index(['Data'], inplace=True)
  return df

Função faz a discretização temporal de dados armazenados em um DataFrame

In [4]:
def discretTemp(df):
  # Resample do dataframe 'df' para obter estatísticas mensais da coluna 'Chuva'
  df_mes1 = df.resample('M').agg({'Chuva': [('Soma', 'sum'), ('Média','mean'), ('Desvio Padrão','std'), ('Máxima','max'), ('Falhas(%)', lambda x: x.isna().sum())]})['Chuva']

  # Resample do dataframe 'df' para contar a quantidade de valores igual a 2 na coluna 'NivelConsistencia' para cada mês
  df_mes2 = df.resample('M').agg({'NivelConsistencia': [('Consistidos(%)', lambda x: (x == 2).sum())]})['NivelConsistencia']

  # Mesclar os dataframes 'df_mes1' e 'df_mes2' com base nos índices (datas)
  df_mes = df_mes1.merge(df_mes2, left_index=True, right_index=True)

  # Calcular a porcentagem de falhas em relação ao número de dias de cada mês
  df_mes['Falhas(%)'] = round(100 * df_mes['Falhas(%)'] / df_mes.index.day, 2)

  # Calcular a porcentagem de valores consistentes em relação ao número de dias de cada mês
  df_mes['Consistidos(%)'] = round(100 * df_mes['Consistidos(%)'] / df_mes.index.day, 2)

  # Resample do dataframe 'df' para obter estatísticas anuais da coluna 'Chuva'
  df_ano1 = df.resample('Y').agg({'Chuva': [('Soma', 'sum'), ('Média','mean'), ('Desvio Padrão','std'), ('Máxima','max'), ('Falhas(%)', lambda x: x.isna().sum())]})['Chuva']

  # Resample do dataframe 'df' para contar a quantidade de valores igual a 2 na coluna 'NivelConsistencia' para cada ano
  df_ano2 = df.resample('Y').agg({'NivelConsistencia': [('Consistidos(%)', lambda x: (x == 2).sum())]})['NivelConsistencia']

  # Mesclar os dataframes 'df_ano1' e 'df_ano2' com base nos índices (datas)
  df_ano = df_ano1.merge(df_ano2, left_index=True, right_index=True)

  # Calcular a porcentagem de falhas em relação ao número de dias do ano (dayofyear)
  df_ano['Falhas(%)'] = round(100 * df_ano['Falhas(%)'] / df_ano.index.dayofyear, 2)

  # Calcular a porcentagem de valores consistentes em relação ao número de dias do ano (dayofyear)
  df_ano['Consistidos(%)'] = round(100 * df_ano['Consistidos(%)'] / df_ano.index.dayofyear, 2)
  return df_mes, df_ano

Função para gerar gráfico a partir de DataFrame

In [5]:
def grafico(dataframe, a, b):
    # Cria uma nova figura do tipo Bar do Plotly
    fig = go.Figure(data=[
        # Adiciona um gráfico de barras no eixo x com base no intervalo de datas de 'a' a 'b'
        go.Bar(x=dataframe.loc[a:b].index, y=dataframe.loc[a:b]['Soma'],
               # Adiciona texto de dica ao passar o mouse sobre cada barra, exibindo a porcentagem de falhas
               hovertext=[str(x) + '%' for x in dataframe.loc[a:b]['Falhas(%)'].tolist()])
    ])

    # Define o modo de agrupamento das barras no gráfico
    fig.update_layout(barmode='group')

    # Mostra o gráfico
    fig.show()

Função para salvar DataFrames em arquivos CSV

In [6]:
def DFtoCSV(df,nome):
  # Caminho completo para o arquivo CSV
  caminho_para_pasta = '/content/drive/MyDrive/ICD/Colab/Alexandre'
  caminho_para_arquivo = f'/{nome}.csv'

  # Exportar o DataFrame para o arquivo CSV
  df_ano.to_csv(caminho_para_pasta+caminho_para_arquivo, index_label='Data')

Visualização do DataFrame por data

In [7]:
caminho_para_pasta = '/content/drive/MyDrive/ICD/Colab/Alexandre'
caminho_para_arquivo = '/chuvas_C_00935056.csv'

df = preparaDF(caminho_para_pasta , caminho_para_arquivo)

periodo = df.loc['2000-12-01':'2000-12-10']
periodo

,NivelConsistencia,Chuva
Data,,
2000-12-01,2,0.0
2000-12-02,2,0.0
2000-12-03,2,0.0
2000-12-04,2,0.0
2000-12-05,2,0.0
2000-12-06,2,0.0
2000-12-07,2,0.0
2000-12-08,2,0.0
2000-12-09,2,0.0


Armazenamento da discretização em DataFrames. Chamando o DataFrame 'df_ano' é possível visualizar valores estatísticos dos dados armazenados ao longo dos anos. O funcionamento é análogo para o DataFrame 'df_mes'.

In [8]:
df_mes = discretTemp(df)[0]
df_ano = discretTemp(df)[1]
df_ano

,Soma,Média,Desvio Padrão,Máxima,Falhas(%),Consistidos(%)
Data,,,,,,
1989-12-31,565.3,4.187407,9.741360,51.0,13.42,41.92
1990-12-31,1409.8,3.862466,8.842641,57.4,0.00,100.00
1991-12-31,1647.5,4.513699,10.025524,88.0,0.00,75.62
1992-12-31,1797.3,4.910656,10.198317,75.5,0.00,91.53
1993-12-31,960.1,2.630411,8.484011,86.0,0.00,58.08
1994-12-31,2012.9,5.514795,12.135594,101.1,0.00,41.10
1995-12-31,1619.1,4.435890,11.410854,94.6,0.00,67.12
1996-12-31,1937.1,5.292623,13.293683,126.9,0.00,33.06
1997-12-31,1668.2,4.570411,12.520640,162.8,0.00,41.10


Chama a função gráfico, podendo utilizar discretização mensal ou anual

In [9]:
#mes = df_mes.loc[df_mes.index.month == 6]
grafico(df_ano,'1989-01-01','2022-12-31')

Avaliando o gráfico, é possível notar que o quanto chove no local da estação tende a respeitar uma curva senoidal com picos próximos dos 2000mm e vales com 1000mm (A estação não marcou no ano de 1989 inteiro). Temos alguns pontos fora da curva, como os anos de 1993, 2000, 2004, 2012, 2017 e 2020. Uma explicação para esses pontos fora da curva pode ser a atuação do El Niño no Brasil que provoca secas em algumas áreas e o aumento das chuvas em outras, ocasionando profundas alterações meteorológicas.

Salva o DataFrame em um arquivo CSV

In [10]:
nome = 'exemplo'
DFtoCSV(df_ano,nome)

In [12]:
periodo = df.loc['2000-01-01':'2001-01-01']
periodo
# Exibe as estatísticas do periodo
periodo_stats = periodo.describe()

print("Estatísticas do periodo:")
print(periodo_stats)
print()

Estatísticas do periodo:
       NivelConsistencia       Chuva
count         367.000000  367.000000
mean            1.667575    6.205177
std             0.471726   15.358400
min             1.000000    0.000000
25%             1.000000    0.000000
50%             2.000000    0.000000
75%             2.000000    6.150000
max             2.000000  199.000000

